# Locust Load Testing

## Install required packages

Below we install required Python packages.

In [ ]:
!pip install --upgrade pip
!pip install locust
!pip install datasets
!which locust

## Locust Testing

Below we do locust testing with a local triton server deployment.

In [ ]:
import subprocess
import time
import os
import stat

ts = round(time.time() * 1000)

os.environ["PROMPT_MODULE_DIR"] = "modules/thudm-longbench"
os.environ["PROMPT_MODULE_NAME"] = "encoder_llm_prompt"
os.environ["PROMPT_GENERATOR_NAME"] = "TextInputGenerator"
os.environ["CONTENT_TYPE"]="application/json"
os.environ["ENDPOINT_NAME"] = "http://localhost:8080/v2/models/model/generate"
os.environ["USERS"]="64"
os.environ["WORKERS"]="8"
os.environ["RUN_TIME"]="60s"
os.environ["SPAWN_RATE"]="64"
os.environ["SCRIPT"]="endpoint_user.py"
results_locust_path = os.path.join("output", "locust-testing")
os.environ["RESULTS_PREFIX"]=f"{results_locust_path}/results-{ts}"
    
try:
    with open("run_locust.log", "w") as logfile:
        print(f"Start Locust testing; logfile: run_locust.log; results: {results_locust_path}")
        path = os.path.join(os.getcwd(), "run_locust.sh")
        os.chmod(path, stat.S_IRUSR | stat.S_IEXEC)
        process = subprocess.Popen(path, encoding="utf-8", 
                                shell=True,stdout=logfile,stderr=subprocess.STDOUT)
        process.wait()
        logfile.flush()
        print(f"Locust testing completed")
except Exception as e:
    print(f"exception occurred: {e}")


## Visualize Locust Results

Below we first visualize the results of the Locust testing in a tabel. 

In [ ]:
import pandas as pd
from IPython.display import display
import numpy as np

results_path = os.environ["RESULTS_PREFIX"] + "_stats.csv"
df = pd.read_csv(results_path)
df = df.replace(np.nan, '')

top_n = 1
caption=f"Locust results"
df = df.truncate(after=top_n - 1, axis=0)
df = df.style \
      .format(precision=6) \
        .set_properties(**{'text-align': 'left'}) \
        .set_caption(caption)
display(df)